In [ ]:
#| default_exp trainer

# Trainer
> Trainer API

In [ ]:
# | export
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import Trainer
import os.path as osp
from loguru import logger



def get_trainer(exp_name, max_epochs, 
                gpus=1,
                monitor=dict(metric="val_acc", mode="max"), 
                save_every_n_epochs=1, 
                save_top_k=1, 
                use_version=True, 
                strategy=None,
                refresh_rate=5):
    """ Example:

    """
    
    
    rld = osp.join("lightning_logs", exp_name)
    cur_num_exps = len(os.listdir(rld)) if osp.exists(rld) else 0
    version = f"{cur_num_exps:02d}"
    filename = "{epoch}-{"+monitor["metric"]+":.2f}"
    root_log_dir = osp.join("lightning_logs", exp_name, version)
    logger.info(f'Log root dir: {root_log_dir}')

    callback_ckpt = ModelCheckpoint(
        dirpath=osp.join(root_log_dir, "ckpts"),
        monitor=monitor['metric'], mode=monitor['mode'],
        filename=filename,
        save_last=True,
        every_n_epochs=save_every_n_epochs,
        save_top_k=save_top_k,
    )

    callback_tqdm = TQDMProgressBar(refresh_rate=5)
    callback_lrmornitor = LearningRateMonitor(logging_interval="step")
    
    plt_logger = TensorBoardLogger(
        osp.join(root_log_dir, "tb_logs"),
    )
    
    if strategy is None:
        strategy="dp" if gpus < 2 else "ddp"
    trainer = Trainer(
        accelerator ='gpu',
        devices=gpus,
        max_epochs=max_epochs,
        strategy=strategy,
        callbacks=[callback_ckpt, callback_tqdm, callback_lrmornitor],
        logger=plt_logger,
    )
    return trainer

In [ ]:
#|hide
get_trainer('test', 10)

2022-09-12 03:23:28.872 | INFO     | __main__:get_trainer:29 - Log root dir: lightning_logs/test/00
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


<pytorch_lightning.trainer.trainer.Trainer>

In [ ]:
#| export
import os
import os.path as osp
# # from argparse import ArgumentParser
# from pytorch_lightning import Trainer, seed_everything
# # from ple.all import get_trainer, BaseExp
import shutil
from fastcore.script import *
from fastcore.utils import *
from loguru import logger

def get_rank() -> int:
    import torch.distributed as dist
    if not dist.is_available():
        return 0
    if not dist.is_initialized():
        return 0
    return dist.get_rank()

def get_exp_by_file(exp_file):
    """
        Params:
        exp_file: Path to exp
        
    """
    try:
        import importlib
        import os
        import sys
        sys.path.append(os.path.dirname(exp_file))
        # import ipdb; ipdb.set_trace()
        current_exp = importlib.import_module(
            os.path.basename(exp_file).split(".")[0])
        current_exp = importlib.reload(current_exp)
        exp = current_exp.Exp()
        return exp
    except Exception:
        raise ImportError(
            "{} doesn't contains class named 'Exp'".format(exp_file))

        
@call_parse
def train(
    cfg_path:Param('Path to config'),
    devices: Param('GPUS indices', default=1, type=int),
    accelerator: Param('cpu or gpu', default='gpu', type=str),
    opts: Param('Additional configs', default='', type=str, required=False),
):
    cfg = get_exp_by_file(cfg_path)
    if len(opts):
        cfg.merge(opts.replace('=',' ').split(' '))
    cfg.devices = devices
    
    data = cfg.get_data_loader()

    model = cfg.get_model(create_lr_scheduler_fn=cfg.get_lr_scheduler(), 
            create_optimizer_fn=cfg.get_optimizer())
    trainer = cfg.get_trainer(devices)
    try:
        trainer.fit(model, data)
    except Exception as e:
        import traceback
        traceback.print_exc()
    finally:
        if get_rank() == 0:
            out_path = osp.join(trainer.log_dir, osp.basename(cfg_path))
            logger.info('cp {} {}', cfg_path, out_path)
            shutil.copy(cfg_path, out_path)
        


# Helps

In [ ]:
example()


    create_schedule_fn = fn_schedule_cosine_with_warmpup_decay_timm(
        num_epochs=EPOCHS,
        num_steps_per_epoch=len(dl_train),
        num_epochs_per_cycle=EPOCHS//2,
        min_lr=1/100,
        cycle_decay=0.7,
    )

    model = Model()
    lit_lstm = LitLSTM(model,create_optimizer_fn=lambda params:torch.optim.Adam(params),
                                   create_lr_scheduler_fn=create_schedule_fn, loss_fn=nn.L1Loss()
                )
    trainer = get_trainer('lstm_trainer', EPOCHS, 
                             monitor={'metric': 'val_loss', 'mode': 'min'},
                         )


## Export

In [ ]:
# get_trainer('test', 10)

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()